<a href="https://colab.research.google.com/github/BartoszJanJerzy/stats/blob/main/stata_AW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#WELCOME!

I (Bartosz Wójtowicz, The John Paul II Catholic University of Lublin) wrote this this code to calculate results in one of master thesis.

---

What is the scehdule?

1 - import libraries

2 - functions used later

3 - import data and screening

4 - research sample

5 - descriptive stats

6 - knowledge questions

7 - main analysis: linear regressions

8 - extra analysis: curvilinear regressions

---

We share this code with you, feel free to use or check it.

#1 - LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import pearsonr, ttest_ind, kurtosis, skew, shapiro, f_oneway
import statistics
from statsmodels.formula.api import ols
from scipy import stats

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



#2 - FUNCTIONS

In [2]:
def KnowledgeQuest(column, labels, title, colors):
  
  # calcualte values
  values = data[column].value_counts(sort=False).to_list()
  
  # calculate widths of bars
  widths = []
  for i in range(len(labels)):
    widths.append(0.5)

  # create graph
  fig = make_subplots(
      rows=2, cols=1,
      specs=[[{"type": "bar"}],
             [{"type": "table"}]]
  )

  # add bar plot
  fig.add_trace(
      go.Bar(
          x=values,
          y=labels,
          orientation='h',
          width=widths,
          marker_color = colors
      ),
      row=1,
      col=1
  )

  # add table
  fig.add_trace(
      go.Table(
          header=dict(
              values=labels,
              font=dict(size=10),
              align="left"
          ),
          cells=dict(
              values=values,
              align = "left")
      ),
      row=2, col=1
  )

  fig.update_layout(
      width=1300,
      font_family='Verdana',
      title_text = title
  )

  return values, fig

In [3]:
def KnowledgeMultipleChoice(values, labels, width, colors, title):
  # create graph
  fig = make_subplots(
      rows=2, cols=1,
      specs=[[{"type": "pie"}],
              [{"type": "table"}]]
  )

  # add bar plot
  fig.add_trace(
      go.Pie(
          values=values,
          labels=labels,
          pull=[0.1, 0.1, 0.1, 0.1, 0.1],
          marker_colors=colors
      ),
      row=1,
      col=1
  )

  # add table
  fig.add_trace(
      go.Table(
          header=dict(
              values=labels,
              font=dict(size=10),
              align="left"
          ),
          cells=dict(
              values=values,
              align = "left")
      ),
      row=2, col=1
  )

  fig.update_layout(
      width=width,
      font_family='Verdana',
      title_text = title
  )

  return fig

In [4]:
def DummyRecodeColumns(df,column, correct):
  df[column] = np.where(df[column] == correct, 1, 0)

In [5]:
def Eta2(df, dep_col, group_col, n_groups):
  SBB = 0
  SSW = 0
  eta2 = 0
  M = df[dep_col].mean()

  for i in range(1, n_groups+1):
    SBB += len(df[dep_col][df[group_col] == i]) * (M - df[dep_col][df[group_col] == i].mean())**2
    SSW += len(df[dep_col][df[group_col] == i]) * df[dep_col][df[group_col] == i].var()
  
  eta2 = SBB/SSW

  return eta2

In [6]:
def CohensD(df1, df2):
  m1 = df1.mean()
  m2 = df2.mean()
  sd = np.sqrt((df1.std()*df1.std() + df2.std()*df2.std()) / 2)

  d = (m1-m2) / sd

  return d

#3 - DATA

##Load data

In [7]:
raw_data = pd.read_excel('data.xlsx')
data = raw_data.copy()
# data.drop('Sygnatura czasowa', axis=1, inplace=True)
data.head()

,Sygnatura czasowa,wiek,miejsce_zamieszkania,wyksztalcenie,stan_cywilny,aktywnosc_zawodowa,wyk_medyczne,porody,porody_typ,mocz_rodzina,mocz_ja,mocz_pomoc,mocz_pomoc_otw,p1,p2,p3,p4,p5,p6_kichanie,p6_kaszel,p6_wysilek,p6_spacer,p6_schody,p7,p8_oslabienie,p8_porody,p8_niskistatus,p8_infekcje,p8_nadwaga,p8_papierosy,p9,p10,p11,p12,p13,p14,p15,p16,p17_higiena,p17_leczeniewad,p17_masaciala,p17_unikaniezaparc,p17_cwiczenie,p18,p19,p20,izz_1,izz_2,izz_3,izz_4,izz_5,izz_6,izz_7,izz_8,izz_9,izz_10,izz_11,izz_12,izz_13,izz_14,izz_15,izz_16,izz_17,izz_18,izz_19,izz_20,izz_21,izz_22,izz_23,izz_24,izz_25
0,2021/01/10 10:11:27 PM CET,2,2,2,1,1,1,1,4,1,1,NaN,NaN,1,1,1,1,1,1,1,1,1,1,3,1,1,0,1,0,0,1,1,1,2,2,1,1,1,0,1,0,0,1,1,1,3.0,4,4,4,4,3,5,3,3,3,4,3,3,2,3,4,4,4,3,3,5,3,4,4,3,NaN
1,2021/01/10 10:16:29 PM CET,2,1,2,2,1,0,3,1,1,2,1.0,3.0,3,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,1,1,1,1,1,1,2,0,1,0,0,1,1,1,2.0,4,5,3,2,3,4,3,3,3,4,3,1,2,2,4,3,4,2,3,5,4,4,5,2,NaN
2,2021/01/10 10:17:21 PM CET,2,1,2,2,1,0,3,1,4,1,1.0,3.0,4,1,1,1,1,1,1,1,0,0,1,1,1,0,1,0,0,1,1,1,2,2,1,1,2,1,1,1,1,1,1,1,1.0,5,3,5,2,2,3,1,2,4,4,2,2,3,4,5,4,4,5,2,5,5,5,2,3,NaN
3,2021/01/10 10:20:32 PM CET,2,2,2,2,1,1,1,4,3,1,NaN,NaN,1,1,1,2,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,3,2,2,1,1,1,0,0,1,1,1,2,1,2.0,3,4,5,4,2,4,3,3,4,4,4,3,4,2,4,4,4,4,3,5,4,5,4,4,NaN
4,2021/01/10 10:23:32 PM CET,2,2,2,2,1,1,2,1,3,2,1.0,4.0,1,1,1,1,1,1,0,1,1,0,3,1,1,0,0,1,0,1,1,1,1,2,1,1,2,1,1,0,0,1,2,1,3.0,4,4,3,3,3,5,3,2,3,3,3,3,4,4,4,2,4,3,3,4,4,5,4,4,NaN


##Make subscales

In [8]:
data['izz_nawyki'] = data['izz_1'] + data['izz_5'] + data['izz_9'] + data['izz_13'] + data['izz_17'] + data['izz_21']
data['izz_profi'] = data['izz_2'] + data['izz_6'] + data['izz_10'] + data['izz_14'] + data['izz_18'] + data['izz_22']
data['izz_psych'] = data['izz_3'] + data['izz_7'] + data['izz_11'] + data['izz_15'] + data['izz_19'] + data['izz_23']
data['izz_zdrow'] = data['izz_4'] + data['izz_8'] + data['izz_12'] + data['izz_16'] + data['izz_20'] + data['izz_24']
data[['izz_nawyki','izz_profi','izz_psych','izz_zdrow']]

,izz_nawyki,izz_profi,izz_psych,izz_zdrow
0,19,23,21,22
1,20,21,21,16
2,23,24,17,18
3,21,23,23,23
4,22,24,20,18
...,...,...,...,...
110,23,25,20,20
111,18,20,21,22
112,20,20,22,21
113,18,22,21,20


In [9]:
data['izz_wo'] = data[['izz_nawyki','izz_profi','izz_psych','izz_zdrow']].sum(axis=1)
data.to_excel('data.xlsx')

In [10]:
print(f'Now - before screening - the sample has N={len(data)} records')
print(f'W tym momencie - przed czyszczeniem danych - próba badawcza posiada N={len(data)} rekordów')

Now - before screening - the sample has N=115 records
W tym momencie - przed czyszczeniem danych - próba badawcza posiada N=115 rekordów


##Screenning

###Calculate means and stds

In [11]:
mean_nawyki = data['izz_nawyki'].mean()
sd_nawyki = data['izz_nawyki'].std()

mean_profi = data['izz_profi'].mean()
sd_profi = data['izz_profi'].std()

mean_psych = data['izz_psych'].mean()
sd_psych = data['izz_psych'].std()

mean_zdrow = data['izz_zdrow'].mean()
sd_zdrow = data['izz_zdrow'].std()

###Make filters

In [12]:
filter_1 = data['izz_nawyki'] > mean_nawyki - 3*sd_nawyki
filter_2 = data['izz_nawyki'] < mean_nawyki + 3*sd_nawyki
filter_3 = data['izz_profi'] > mean_profi - 3*sd_profi
filter_4 = data['izz_profi'] < mean_profi + 3*sd_profi
filter_5 = data['izz_psych'] > mean_psych - 3*sd_psych
filter_6 = data['izz_psych'] < mean_psych + 3*sd_psych
filter_7 = data['izz_zdrow'] > mean_zdrow - 3*sd_zdrow
filter_8 = data['izz_zdrow'] < mean_zdrow + 3*sd_zdrow

###Use filters

In [13]:
data = data[filter_1]
data = data[filter_2]
data = data[filter_3]
data = data[filter_4]
data = data[filter_5]
data = data[filter_6]
data = data[filter_7]
data = data[filter_8]

print(f'Now - after screening - the sample has N={len(data)} records')
print(f'W tym momencie - po czyszczeniem danych - próba badawcza posiada N={len(data)} rekordów')

Now - after screening - the sample has N=115 records
W tym momencie - po czyszczeniem danych - próba badawcza posiada N=115 rekordów


#4 - RESEARCH SAMPLE

##Converse numbers into labels

In [14]:
df_demo = data[['wiek','miejsce_zamieszkania','wyksztalcenie','stan_cywilny',	'aktywnosc_zawodowa', 'wyk_medyczne',
                'porody','porody_typ','mocz_rodzina','mocz_ja','mocz_pomoc', 'mocz_pomoc_otw']]

for i in range(len(df_demo)):
  # wiek
  if df_demo['wiek'][i] == 1:
    df_demo['wiek'][i] = '20-25 lat'
  elif df_demo['wiek'][i] == 2:
    df_demo['wiek'][i] = '26-35 lat'
  elif df_demo['wiek'][i] == 3:
    df_demo['wiek'][i] = '36-50 lat'
  elif df_demo['wiek'][i] == 4:
    df_demo['wiek'][i] = 'pow. 50 lat'

  # miejsce zamieszkania
  if df_demo['miejsce_zamieszkania'][i] == 1:
    df_demo['miejsce_zamieszkania'][i] = 'Wieś'
  elif df_demo['miejsce_zamieszkania'][i] == 2:
    df_demo['miejsce_zamieszkania'][i] = 'Miasto'
  
  # wykształcenie
  if df_demo['wyksztalcenie'][i] == 1:
    df_demo['wyksztalcenie'][i] = 'Średnie'
  elif df_demo['wyksztalcenie'][i] == 2:
    df_demo['wyksztalcenie'][i] = 'Wyższe'
  
  # stan cywilny
  if df_demo['stan_cywilny'][i] == 1:
    df_demo['stan_cywilny'][i] = 'Panna'
  elif df_demo['stan_cywilny'][i] == 2:
    df_demo['stan_cywilny'][i] = 'Mężatka'
  elif df_demo['stan_cywilny'][i] == 3:
    df_demo['stan_cywilny'][i] = 'Wdowa'
  elif df_demo['stan_cywilny'][i] == 4:
    df_demo['stan_cywilny'][i] = 'Rozwiedziona'
  
  # aktywność zawodowa
  if df_demo['aktywnosc_zawodowa'][i] == 1:
    df_demo['aktywnosc_zawodowa'][i] = 'Aktywna zawodowo'
  elif df_demo['aktywnosc_zawodowa'][i] == 2:
    df_demo['aktywnosc_zawodowa'][i] = 'Aktualnie nie pracuję'
  elif df_demo['aktywnosc_zawodowa'][i] == 3:
    df_demo['aktywnosc_zawodowa'][i] = 'Rencistka/emerytka'
  
  # wykształcenie medyczne
  if df_demo['wyk_medyczne'][i] == 1:
    df_demo['wyk_medyczne'][i] = 'Tak'
  elif df_demo['wyk_medyczne'][i] == 0:
    df_demo['wyk_medyczne'][i] = 'Nie'
  
  # porody
  if df_demo['porody'][i] == 1:
    df_demo['porody'][i] = 'Nie rodziłam'
  elif df_demo['porody'][i] == 2:
    df_demo['porody'][i] = '1 poród'
  elif df_demo['porody'][i] == 3:
    df_demo['porody'][i] = '2 porody'
  elif df_demo['porody'][i] == 4:
    df_demo['porody'][i] = 'więcej niż 2 razy'

  # porody typ
  if df_demo['porody_typ'][i] == 1:
    df_demo['porody_typ'][i] = 'Tylko naturalnie'
  elif df_demo['porody_typ'][i] == 2:
    df_demo['porody_typ'][i] = 'Tylko cięciem cesarskim'
  elif df_demo['porody_typ'][i] == 3:
    df_demo['porody_typ'][i] = 'Naturalnie i cięciem cesarskim'
  elif df_demo['porody_typ'][i] == 4:
    df_demo['porody_typ'][i] = 'Nie'
  
  # mocz rodzina
  if df_demo['mocz_rodzina'][i] == 1:
    df_demo['mocz_rodzina'][i] = 'nie'
  elif df_demo['mocz_rodzina'][i] == 2:
    df_demo['mocz_rodzina'][i] = 'u siostry'
  elif df_demo['mocz_rodzina'][i] == 3:
    df_demo['mocz_rodzina'][i] = 'u mamy'
  elif df_demo['mocz_rodzina'][i] == 3:
    df_demo['mocz_rodzina'][i] = 'u babci'
  
  # mocz ja
  if df_demo['mocz_ja'][i] == 1:
    df_demo['mocz_ja'][i] = 'nie'
  elif df_demo['mocz_ja'][i] == 2:
    df_demo['mocz_ja'][i] = 'kilka razy'
  elif df_demo['mocz_ja'][i] == 3:
    df_demo['mocz_ja'][i] = 'regularnie'

  # pomoc medyczna
  if df_demo['mocz_pomoc'][i] == 1:
    df_demo['mocz_pomoc'][i] = 'nie'
  elif df_demo['mocz_pomoc'][i] == 2:
    df_demo['mocz_pomoc'][i] = 'tak'

  # pomoc medyczna dlaczego nie
  if df_demo['mocz_pomoc_otw'][i] == 1:
    df_demo['mocz_pomoc_otw'][i] = 'Nie wiem, gdzie mam się zgłosić z  tym problemem'
  elif df_demo['mocz_pomoc_otw'][i] == 2:
    df_demo['mocz_pomoc_otw'][i] = 'Wstydzę się rozmawiać na ten temat'
  elif df_demo['mocz_pomoc_otw'][i] == 3:
    df_demo['mocz_pomoc_otw'][i] = 'Uważam, że problem ten nie wymaga konsultacji z lekarzem'
  elif df_demo['mocz_pomoc_otw'][i] == 4:
    df_demo['mocz_pomoc_otw'][i] = 'Inne'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1009: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning:


##Show demographics

In [15]:
categories = ['20-25 lat','26-35 lat','36-50 lat','pow. 50 lat',
              'Wieś','Miasto',
              'Średnie','Wyższe',
              'Panna','Mężatka','Wdowa','Rozwiedziona',
              'Aktywna zawodowo','Aktualnie nie pracuję','Rencistka/emerytka',
              'Tak','Nie',
              'Nie rodziłam','1 poród','2 porody','3 porody',
              'Tylko naturalnie','Tylko cięciem cesarskim','Naturalnie i cięciem cesarskim',
              'Nie','U siostry','u mamy','u babci',
              'regularnie','nie','kilka razy',
              'tak','nie',
              'Nie wiem, gdzie mam się zgłosić z  tym problemem', 'Wstydzę się rozmawiać na ten temat', 'Uważam, że problem ten nie wymaga konsultacji z lekarzem', 'Inne'
              ]
values_1 = df_demo['wiek'].value_counts(sort=False)
values_2 = df_demo['miejsce_zamieszkania'].value_counts(sort=False).to_list()
values_3 = df_demo['wyksztalcenie'].value_counts(sort=False).to_list()
values_4 = df_demo['stan_cywilny'].value_counts(sort=False).to_list()
values_5 = df_demo['aktywnosc_zawodowa'].value_counts(sort=False).to_list()
values_6 = df_demo['wyk_medyczne'].value_counts(sort=False).to_list()
values_7 = df_demo['porody'].value_counts(sort=False).to_list()

filter_temp = df_demo['porody'] != 'Nie rodziłam'
df_demo_temp = df_demo[filter_temp]
values_8 = df_demo_temp['porody_typ'].value_counts(sort=False).to_list()

values_9 = df_demo['mocz_rodzina'].value_counts(sort=False).to_list()
values_10 = df_demo['mocz_ja'].value_counts(sort=False).to_list()

filter_temp_2 = df_demo['mocz_ja'] != 'nie'
df_demo_temp_2 = df_demo[filter_temp_2]
values_11 = df_demo_temp_2['mocz_pomoc'].value_counts(sort=False).to_list()

filter_temp_3 = df_demo_temp_2['mocz_pomoc'] == 'nie'
df_demo_temp_3 = df_demo_temp_2[filter_temp_3]
values_12 = df_demo_temp_3['mocz_pomoc_otw'].value_counts(sort=False).to_list()


values = []
percentages = []
j = 1

for results in [values_1,values_2,values_3,values_4,values_5,values_6, values_7, values_8, values_9, values_10, values_11, values_12]:
  if j != 8 and j != 12 and j!= 11:
    for i in results:
      values.append(i)
      percentages.append(str(round((100*i) / len(df_demo),1)) + '%')
  elif j == 8:
    for i in results:
      values.append(i)
      percentages.append(str(round((100*i) / len(df_demo_temp),1)) + '%')
  elif j == 11:
    for i in results:
      values.append(i)
      percentages.append(str(round((100*i) / len(df_demo_temp_2),1)) + '%')
  elif j == 12:
    for i in results:
      values.append(i)
      percentages.append(str(round((100*i) / len(df_demo_temp_3),1)) + '%')
  
  j += 1


dict_sample_demo = {
    'cat':categories,
    'N':values,
    '%':percentages
}

sample_demo_desc = pd.DataFrame(dict_sample_demo)
sample_demo_desc

,cat,N,%
0,20-25 lat,14,12.2%
1,26-35 lat,7,6.1%
2,36-50 lat,51,44.3%
3,pow. 50 lat,43,37.4%
4,Wieś,65,56.5%
5,Miasto,50,43.5%
6,Średnie,11,9.6%
7,Wyższe,104,90.4%
8,Panna,80,69.6%
9,Mężatka,27,23.5%


In [16]:
sample_demo_desc.to_excel('demo_stats.xlsx')

#5 - CORRECTNESS OF KNOWLEDGE ANSWERS

##Data

In [17]:
data_test = data[['p1','p2','p3','p5','p6_kichanie','p6_kaszel','p6_wysilek','p6_spacer','p6_schody','p7',
                  'p8_oslabienie','p8_porody','p8_niskistatus','p8_infekcje','p8_nadwaga','p8_papierosy',
                  'p9','p10','p12','p13','p14','p15','p16',
                  'p17_higiena','p17_leczeniewad','p17_masaciala','p17_unikaniezaparc','p17_cwiczenie',
                  'p18']]
data_points = data_test.copy()

In [18]:
len(data_points.columns)

29

##Recode to points

In [19]:
DummyRecodeColumns(data_points, 'p1', 1)
DummyRecodeColumns(data_points, 'p3', 1)
DummyRecodeColumns(data_points, 'p5', 1)
DummyRecodeColumns(data_points, 'p7', 1)
DummyRecodeColumns(data_points, 'p9', 1)
DummyRecodeColumns(data_points, 'p10', 1)
DummyRecodeColumns(data_points, 'p12', 2)
DummyRecodeColumns(data_points, 'p13', 2)
DummyRecodeColumns(data_points, 'p14', 1)
DummyRecodeColumns(data_points, 'p15', 1)
DummyRecodeColumns(data_points, 'p16', 1)

##Calculate points

In [20]:
points = data_points.sum(axis=1)

In [21]:
data['points'] = points

#6 - DESCRIPTIVE STATS

In [22]:
desc_stats = data[['izz_nawyki','izz_profi','izz_psych','izz_zdrow']]
desc_stats['izz_wo'] = desc_stats.sum(axis=1)
desc_stats['points'] = data['points']
desc_stats

k = []
sk = []
sh = []

for c in desc_stats.columns:
  k.append(kurtosis(desc_stats[c]))
  sk.append(skew(desc_stats[c]))
  sh.append(shapiro(desc_stats[c])[1])

desc_stats = desc_stats.describe()
desc_stats = desc_stats.drop(['25%','50%','75%','count'])
desc_stats = desc_stats.T
desc_stats['K'] = k
desc_stats['Sk'] = sk
desc_stats['p'] = sh

desc_stats

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,mean,std,min,max,K,Sk,p
izz_nawyki,20.626087,3.784690,11.0,30.0,-0.082096,-0.022310,0.559336
izz_profi,21.782609,3.986342,12.0,30.0,-0.459511,-0.197824,0.204937
izz_psych,20.704348,4.047902,11.0,29.0,-0.086259,-0.533218,0.001126
izz_zdrow,19.747826,4.092877,9.0,28.0,-0.367887,-0.137809,0.246749
izz_wo,82.860870,12.757445,49.0,114.0,0.164581,-0.306379,0.317617
points,21.000000,3.436134,13.0,29.0,-0.464144,-0.152447,0.113434


In [23]:
desc_stats.to_excel('desc_stats.xlsx')

#7 - KNOWLEDGE QUESTIONS

In [24]:
labels = ['Może wystepować w każdym wieku','U kobiet w ciąży i po porodzie','U kobiet pow. 45 r.ż.','U kobiet podczas lub po menopauzie']
column = 'p1'
title = 'Jakiej grupy kobiet w Pani ocenie dotyczy problem nietrzymania moczu?'
colors = ['#38B000','#E35053','#DD2C2F','#D02224']
 
values, fig= KnowledgeQuest(column, labels, title, colors)
fig.show()

In [25]:
'''
NA PYTANIE 2 WSZYSTKIE OSOBY ODPWOEIDZIAŁY TAK SAMO
'''

'\nNA PYTANIE 2 WSZYSTKIE OSOBY ODPWOEIDZIAŁY TAK SAMO\n'

In [26]:
labels = ['urologa i ginekologa','chirurga i urologa','internisty i urologa']
column = 'p3'
title = 'Współpraca jakich dwóch specjalności lekarzy obejmuje prawidłowo przeprowadzony proces diagnostyczny?'
colors = ['#38B000','#E35053','#DD2C2F']

values, fig = KnowledgeQuest(column, labels, title, colors)
fig.show()

In [27]:
labels = ['Nie','Tak','Nie wiem']
column = 'p4'
title = 'Czy choroba jest uznawana przez Panią za wstydliwą?'

values, fig = KnowledgeQuest(column, labels, title, '#468FAF')
fig.show()

In [28]:
labels = ['mimowolnym gubieniem moczu w czasie kaszlu,<br>kichania, śmiechu, czy innym wysiłku fizycznym',
          'mimowolnym wycieku moczu<br>poprzedzonym uczuciem nagłego parcia na mocz',
          'odczuwaniu ciągłego parcia na pęcherz']
column = 'p5'
title = 'Wysiłkowe nietrzymanie moczu polega na:'
colors = ['#38B000','#E35053','#DD2C2F']

values, fig = KnowledgeQuest(column, labels, title, colors)
fig.show()

In [29]:
kichanie = data['p6_kichanie'].sum()
kaszel = data['p6_kaszel'].sum()
wysiłek = data['p6_wysilek'].sum()
spacer = data['p6_spacer'].sum()
schody = data['p6_schody'].sum()

values = [kichanie, kaszel, wysiłek, spacer, schody]
labels = ['Kichanie','Kaszel','Wysiłek fizyczny','Spacer','Schody']
colors = ['#007200','#008000','#38B000','#DD2C2F','#E35053']
title = 'Okoliczności sprzyjające pojawieniu się epizodów wysiłkowego nietrzymania moczu to:'

fig = KnowledgeMultipleChoice(values, labels, 900, colors, title)
fig.show()

In [30]:
labels = ['mimowolnym wycieku moczu<br>poprzedzonym uczuciem nagłego parcia na mocz',
          'mimowolnym gubieniem moczu w czasie kaszlu,<br>kichania, śmiechu, czy innym wysiłku fizycznym',
          'odczuwaniu ciągłego parcia na pęcherz']
column = 'p7'
title = 'Naglące nietrzymanie moczu polega na:'
colors = ['#38B000','#E35053','#DD2C2F']

values, fig = KnowledgeQuest(column, labels, title, colors)
fig.show()

In [31]:
oslabienie = data['p8_oslabienie'].sum()
porody = data['p8_porody'].sum()
status = data['p8_niskistatus'].sum()
infekcje = data['p8_infekcje'].sum()
nadwaga = data['p8_nadwaga'].sum()
papierosy = data['p8_papierosy'].sum()


values = [oslabienie, porody, status, infekcje, nadwaga, papierosy]
labels = ['oslabienie','proody','status','infekcje','nadwaga','papierosy']
colors = ['#38B000','#008000','#DD2C2F','#007200','#006400','#E35053']
title = 'Proszę, aby Pani zaznaczyła, które czynniki zwiększają ryzyko<br>wystąpienia problemu nietrzymania moczu?'

fig = KnowledgeMultipleChoice(values, labels, 900, colors, title)
fig.show()

In [32]:
labels = ['Mięśnie dna miednicy (Mięśnie Kegla)', 'Mięśnie boczne i przednie brzucha']
column = 'p9'
title = ' Jakie mięśnie odpowiedzialne są za utrzymanie moczu w pęcherzu<br>i kontrolowanie zwieracza cewki moczowej?'
colors = ['#38B000','#E35053','#DD2C2F']

values, fig = KnowledgeQuest(column, labels, title, colors)
fig.show()

In [33]:
labels = ['Na naprzemiennym zaciskaniu<br>i rozluźnianiu mięśni dna miednicy',
          'Na naprzemiennym  zaciskaniu<br>i rozluźnianiu mięśni pośladków',
          'Na regularnym przerywaniu oddawania moczu do toalety']
column = 'p10'
title = 'Na czym polegają ćwiczenia mięśni Kegla?'
colors = ['#38B000','#E35053','#DD2C2F']

values, fig = KnowledgeQuest(column, labels, title, colors)
fig.show()

In [34]:
labels = ['Nie, nie wykonuję',
          'Tak, kilka razy w tygodniu',
          'Tak, kilka razy w miesiącu',
          'Tak, wykonuję je codziennie']
column = 'p11'
title = 'Czy wykonuje Pani ćwiczenia mięśni Kegla?'

values, fig = KnowledgeQuest(column, labels, title, '#468FAF')
fig.show()

In [35]:
labels = ['Nie wiem','Tak','Nie']
column = 'p12'
title = 'Czy ćwiczenia mięśni Kegla są skuteczne w leczeniu nietrzymania moczu?'
colors = ['#DD2C2F','#38B000']

values, fig = KnowledgeQuest(column, labels, title, colors)
fig.show()

In [36]:
labels = ['Nie wiem','Tak','Nie']
column = 'p13'
title = 'Czy leczenie nietrzymania moczu jest możliwe?'
colors = ['#E35053','#38B000','#DD2C2F']


values, fig = KnowledgeQuest(column, labels, title, colors)
fig.show()

In [37]:
labels = ['leczenie zachowawcze i leczenie operacyjne',
          'hormonoterapia i leczenie operacyjne',
          'nie ma skutecznej metody leczenia nietrzymania moczu']
column = 'p14'
title = 'Dwie główne metody leczenia nietrzymania moczu to:'
colors = ['#38B000','#E35053','#DD2C2F']

values, fig = KnowledgeQuest(column, labels, title, colors)
fig.show()

In [38]:
labels = ['Leczenie zachowawcze','Leczenie operacyjne','Leczenie hormonalne']
column = 'p15'
title = 'Jaka jest najpowszechniejsza, a zarazem najtańsza metoda leczenia nietrzymania moczu?'
colors = ['#38B000','#E35053','#DD2C2F']

values, fig = KnowledgeQuest(column, labels, title, colors)
fig.show()

In [39]:
labels = ['Farmakoterapii, fizjoterapii, terapii behawioralnej',
          'Fizjoterapii, farmakoterapii',
          'Farmakoterapii, zabiegów operacyjnych']
column = 'p16'
title = 'Leczenie zachowawcze nietrzymania moczu składa się z :'
colors = ['#38B000','#E35053','#DD2C2F']

values, fig = KnowledgeQuest(column, labels, title, colors)
fig.show()

In [40]:
higiena = data['p17_higiena'].sum()
leczenie = data['p17_leczeniewad'].sum()
masa = data['p17_masaciala'].sum()
unikanie = data['p17_unikaniezaparc'].sum()
cwiczenie = data['p17_cwiczenie'].sum()

values = [higiena, leczenie, masa, unikanie, cwiczenie]
labels = ['Higienia i zdrowie','Leczenie wad układu moczowo-płciowego','Prawidłowa masa ciała','Unikanie zaparć','Ćwiczenie mięśni dna miednicy']
colors = ['#38B000','#008000','#007200','#006400','#004B23']
title = 'Zapobieganie schorzeniu polega na:'

fig = KnowledgeMultipleChoice(values, labels, 1000, colors, title)
fig.show()

In [41]:
labels = ['Nie, nie jest znane','Tak, jest znane']
column = 'p18'
title = 'Czy znane jest Pani pojęcie fizjoterapii uroginekologicznej?'

values, fig = KnowledgeQuest(column, labels, title, '#468FAF')
fig.show()

In [42]:
labels = ['Nie, nie korzystałam','Tak, korzystałam']
column = 'p19'
title = 'Czy korzystała Pani z pomocy fizjoterapeuty uroginekologicznego?'

values, fig = KnowledgeQuest(column, labels, title, '#468FAF')
fig.show()

In [43]:
labels = ['Niski','Zadowalający','Dobry','Bardzo dobry']
column = 'p20'
title = 'Jak ocenia Pani stan swojej wiedzy na temat nietrzymania moczu? '

values, fig = KnowledgeQuest(column, labels, title, '#468FAF')
fig.show()

#8 - DEMOGRAPHICS IMPACT ON KNOWLEDGE

##H1

In [44]:
fig = go.Figure(
    data=go.Histogram(x=data['points'],
                      marker_color='#468FAF',
                      xbins=dict(size=1)),
    layout=go.Layout(width=1000,
                     yaxis_title_text='Liczba osób',
                     bargap=0.1,
                     title='Liczba poprawnych odpowiedzi w pytaniach dotyczących wiedzy (max=29)',
                     font_family='Verdana')
)

fig.show()

##H2 - age

In [45]:
M = data['points'].mean()
M

21.0

In [46]:
means = data[['wiek','points']].groupby(['wiek']).mean()
stds = data[['wiek','points']].groupby(['wiek']).std()

age_dict = {
    'wiek':['20-25 lat', '26-35 lat', '36-50 lat', 'pow. 50 lat'],
    'M':means['points'].to_list(),
    'SD':stds['points'].to_list()
}

df_age = pd.DataFrame(age_dict)
df_age = df_age.drop([3])
df_age

,wiek,M,SD
0,20-25 lat,20.000000,3.616203
1,26-35 lat,21.651163,3.604783
2,36-50 lat,20.882353,3.216502


In [47]:
f, p = f_oneway(data['points'][data['wiek']==1],
                data['points'][data['wiek']==2],
                data['points'][data['wiek']==3])
eta = Eta2(data, 'points', 'wiek', 3)

In [48]:
df_age['F'] = [f, '-', '-']
df_age['p'] = [p, '-', '-']
df_age['eta2'] = [eta, '-', '-']
df_age

,wiek,M,SD,F,p,eta2
0,20-25 lat,20.000000,3.616203,1.37719,0.256806,0.0259464
1,26-35 lat,21.651163,3.604783,-,-,-
2,36-50 lat,20.882353,3.216502,-,-,-


##H3 - residence

In [49]:
means = data[['miejsce_zamieszkania','points']].groupby(['miejsce_zamieszkania']).mean()
stds = data[['miejsce_zamieszkania','points']].groupby(['miejsce_zamieszkania']).std()

dict_temp = {
    'miejsce':['wieś','miasto'],
    'M':means['points'].to_list(),
    'SD':stds['points'].to_list()
}

df = pd.DataFrame(dict_temp)
df

,miejsce,M,SD
0,wieś,20.640000,3.192849
1,miasto,21.276923,3.612079


In [50]:
t, p = ttest_ind(data['points'][data['miejsce_zamieszkania']==1],
                data['points'][data['miejsce_zamieszkania']==2])
# eta = Eta2(data, 'points', 'miejsce_zamieszkania', 2)

In [51]:
df['t'] = [t, '-',]
df['p'] = [p, '-',]
# df['d'] = [d, '-',]
df

,miejsce,M,SD,t,p
0,wieś,20.640000,3.192849,-0.985267,0.326598
1,miasto,21.276923,3.612079,-,-


##H4 - education

###General

In [52]:
means = data[['wyksztalcenie','points']].groupby(['wyksztalcenie']).mean()
stds = data[['wyksztalcenie','points']].groupby(['wyksztalcenie']).std()

dict_temp = {
    'wyksztalcenie':['średnie','wyższe'],
    'M':means['points'].to_list(),
    'SD':stds['points'].to_list()
}

df = pd.DataFrame(dict_temp)
df

,wyksztalcenie,M,SD
0,średnie,19.454545,3.357488
1,wyższe,21.163462,3.419279


In [53]:
t, p = ttest_ind(data['points'][data['wyksztalcenie']==1],
                data['points'][data['wyksztalcenie']==2])
# eta = Eta2(data, 'points', 'wyksztalcenie', 2)

In [54]:
df['t'] = [t, '-',]
df['p'] = [p, '-',]
# df_miejsce['eta2'] = [eta, '-',]
df

,wyksztalcenie,M,SD,t,p
0,średnie,19.454545,3.357488,-1.57885,0.117167
1,wyższe,21.163462,3.419279,-,-


###Medical ***

In [55]:
means = data[['wyk_medyczne','points']].groupby(['wyk_medyczne']).mean()
stds = data[['wyk_medyczne','points']].groupby(['wyk_medyczne']).std()

dict_temp = {
    'wyksztalcenie':['nie','tak'],
    'M':means['points'].to_list(),
    'SD':stds['points'].to_list()
}

df = pd.DataFrame(dict_temp)
df

,wyksztalcenie,M,SD
0,nie,19.250000,3.228224
1,tak,21.797468,3.243849


In [56]:
t, p = ttest_ind(data['points'][data['wyk_medyczne']==0],
                data['points'][data['wyk_medyczne']==1])

# eta = Eta2(data, 'points', 'wyksztalcenie', 2)

In [57]:
df['t'] = [t, '-',]
df['p'] = [p, '-',]
# df_miejsce['eta2'] = [eta, '-',]
df

,wyksztalcenie,M,SD,t,p
0,nie,19.250000,3.228224,-3.91121,0.000157168
1,tak,21.797468,3.243849,-,-


In [58]:
d = CohensD(data['points'][data['wyk_medyczne']==0], data['points'][data['wyk_medyczne']==1])

In [59]:
df['d'] = [d, '-',]
df

,wyksztalcenie,M,SD,t,p,d
0,nie,19.250000,3.228224,-3.91121,0.000157168,-0.787216
1,tak,21.797468,3.243849,-,-,-


##H5 - marital status

In [60]:
means = data[['stan_cywilny','points']].groupby(['stan_cywilny']).mean()
stds = data[['stan_cywilny','points']].groupby(['stan_cywilny']).std()

dict_temp = {
    'stan cywilny':['Panna','Mężatka','Wdowa','Rozwiedziona'],
    'M':means['points'].to_list(),
    'SD':stds['points'].to_list()
}

df = pd.DataFrame(dict_temp)
df = df.drop([2,3])
df

,stan cywilny,M,SD
0,Panna,20.481481,4.709813
1,Mężatka,21.137500,2.949979


In [61]:
t, p = ttest_ind(data['points'][data['stan_cywilny']==1],
                data['points'][data['stan_cywilny']==2])
# eta = Eta2(data, 'points', 'stan_cywilny', 4)

In [62]:
df['t'] = [t, '-',]
df['p'] = [p, '-',]
# df['eta2'] = [eta, '-', '-', '-']
df

,stan cywilny,M,SD,t,p
0,Panna,20.481481,4.709813,-0.849441,0.397569
1,Mężatka,21.137500,2.949979,-,-


##H6 - work status

In [63]:
means = data[['aktywnosc_zawodowa','points']].groupby(['aktywnosc_zawodowa']).mean()
stds = data[['aktywnosc_zawodowa','points']].groupby(['aktywnosc_zawodowa']).std()

dict_temp = {
    'aktywnosc_zawodowa':['Aktywna zawodowo','Aktualnie nie pracuję', 'Rencistka/emerytka'],
    'M':means['points'].to_list(),
    'SD':stds['points'].to_list()
}

df = pd.DataFrame(dict_temp)
df

,aktywnosc_zawodowa,M,SD
0,Aktywna zawodowo,21.207921,3.259806
1,Aktualnie nie pracuję,20.000000,4.669047
2,Rencistka/emerytka,17.666667,2.886751


In [64]:
data['aktywnosc_zawodowa'].value_counts()

1    101
2     11
3      3
Name: aktywnosc_zawodowa, dtype: int64

##H7 - Births

###Number of births

In [65]:
means = data[['porody','points']].groupby(['porody']).mean()
stds = data[['porody','points']].groupby(['porody']).std()

dict_temp = {
    'porody':['0 porodów','1 poród','2 porody','więcej niż 2 porody'],
    'M':means['points'].to_list(),
    'SD':stds['points'].to_list()
}

df = pd.DataFrame(dict_temp)
# df_age = df_age.drop([3])
df

,porody,M,SD
0,0 porodów,20.738095,4.084829
1,1 poród,21.516129,2.618791
2,2 porody,20.555556,3.166329
3,więcej niż 2 porody,21.466667,3.542934


In [66]:
f, p = f_oneway(data['points'][data['porody']==1],
                data['points'][data['porody']==2],
                data['points'][data['porody']==3],
                data['points'][data['porody']==4])
eta = Eta2(data, 'points', 'porody', 4)

In [67]:
df['F'] = [f, '-', '-', '-']
df['p'] = [p, '-', '-', '-']
df['eta2'] = [eta, '-', '-', '-']
df

,porody,M,SD,F,p,eta2
0,0 porodów,20.738095,4.084829,0.550679,0.648733,0.014383
1,1 poród,21.516129,2.618791,-,-,-
2,2 porody,20.555556,3.166329,-,-,-
3,więcej niż 2 porody,21.466667,3.542934,-,-,-


###Type of births

In [68]:
means = data[['porody_typ','points']][data['porody'] != 1].groupby(['porody_typ']).mean()
stds = data[['porody_typ','points']][data['porody'] != 1].groupby(['porody_typ']).std()

dict_temp = {
    'porody_typ':['Tylko naturalne','Tylko cesarskie cięcie', 'inne'],
    'M':means['points'].to_list(),
    'SD':stds['points'].to_list()
}

df = pd.DataFrame(dict_temp)
df = df.drop([2])
df

,porody_typ,M,SD
0,Tylko naturalne,20.744186,3.055775
1,Tylko cesarskie cięcie,21.560000,2.662705


In [69]:
t, p = ttest_ind(data['points'][data['porody_typ']==1],
                data['points'][data['porody_typ']==2])
# eta = Eta2(data, 'points', 'stan_cywilny', 4)

In [70]:
df['t'] = [t, '-',]
df['p'] = [p, '-',]
# df['eta2'] = [eta, '-', '-', '-']
df

,porody_typ,M,SD,t,p
0,Tylko naturalne,20.744186,3.055775,-1.11125,0.270493
1,Tylko cesarskie cięcie,21.560000,2.662705,-,-


#9 - DEMOGRAPHICS IMPACT ON PRO-HEALTH BEHAVIOURS

##Age

In [73]:
means = data[['wiek','izz_wo']].groupby(['wiek']).mean()
stds = data[['wiek','izz_wo']].groupby(['wiek']).std()

dict_temp = {
    'wiek':['20-25 lat', '26-35 lat', '36-50 lat', 'pow. 50 lat'],
    'M':means['izz_wo'].to_list(),
    'SD':stds['izz_wo'].to_list()
}

df_temp = pd.DataFrame(age_dict)
df_temp = df_temp.drop([3])
df_temp

,wiek,M,SD
0,20-25 lat,20.000000,3.616203
1,26-35 lat,21.651163,3.604783
2,36-50 lat,20.882353,3.216502


In [74]:
f, p = f_oneway(data['izz_wo'][data['wiek']==1],
                data['izz_wo'][data['wiek']==2],
                data['izz_wo'][data['wiek']==3])
eta = Eta2(data, 'izz_wo', 'wiek', 3)

In [75]:
df_temp['F'] = [f, '-', '-']
df_temp['p'] = [p, '-', '-']
df_temp['eta2'] = [eta, '-', '-']
df_temp

,wiek,M,SD,F,p,eta2
0,20-25 lat,20.000000,3.616203,0.120938,0.886212,0.00252532
1,26-35 lat,21.651163,3.604783,-,-,-
2,36-50 lat,20.882353,3.216502,-,-,-


##Residence

In [76]:
means = data[['miejsce_zamieszkania','izz_wo']].groupby(['miejsce_zamieszkania']).mean()
stds = data[['miejsce_zamieszkania','izz_wo']].groupby(['miejsce_zamieszkania']).std()

dict_temp = {
    'miejsce':['wieś','miasto'],
    'M':means['izz_wo'].to_list(),
    'SD':stds['izz_wo'].to_list()
}

df = pd.DataFrame(dict_temp)
df

,miejsce,M,SD
0,wieś,82.480000,14.198649
1,miasto,83.153846,11.634043


In [77]:
t, p = ttest_ind(data['izz_wo'][data['miejsce_zamieszkania']==1],
                data['izz_wo'][data['miejsce_zamieszkania']==2])
# eta = Eta2(data, 'points', 'miejsce_zamieszkania', 2)

In [78]:
df['t'] = [t, '-',]
df['p'] = [p, '-',]
# df['d'] = [d, '-',]
df

,miejsce,M,SD,t,p
0,wieś,82.480000,14.198649,-0.279658,0.780251
1,miasto,83.153846,11.634043,-,-


##Education

In [79]:
means = data[['wyk_medyczne','izz_wo']].groupby(['wyk_medyczne']).mean()
stds = data[['wyk_medyczne','izz_wo']].groupby(['wyk_medyczne']).std()

dict_temp = {
    'wyksztalcenie':['nie','tak'],
    'M':means['izz_wo'].to_list(),
    'SD':stds['izz_wo'].to_list()
}

df = pd.DataFrame(dict_temp)
df

,wyksztalcenie,M,SD
0,nie,83.388889,11.608563
1,tak,82.620253,13.312134


In [80]:
t, p = ttest_ind(data['izz_wo'][data['wyk_medyczne']==0],
                data['izz_wo'][data['wyk_medyczne']==1])

# eta = Eta2(data, 'points', 'wyksztalcenie', 2)

In [81]:
df['t'] = [t, '-',]
df['p'] = [p, '-',]
# df_miejsce['eta2'] = [eta, '-',]
df

,wyksztalcenie,M,SD,t,p
0,nie,83.388889,11.608563,0.298422,0.765929
1,tak,82.620253,13.312134,-,-


##Marital status

In [82]:
means = data[['stan_cywilny','izz_wo']].groupby(['stan_cywilny']).mean()
stds = data[['stan_cywilny','izz_wo']].groupby(['stan_cywilny']).std()

dict_temp = {
    'stan cywilny':['Panna','Mężatka','Wdowa','Rozwiedziona'],
    'M':means['izz_wo'].to_list(),
    'SD':stds['izz_wo'].to_list()
}

df = pd.DataFrame(dict_temp)
df = df.drop([2,3])
df

,stan cywilny,M,SD
0,Panna,83.148148,11.561553
1,Mężatka,82.350000,12.737963


In [83]:
t, p = ttest_ind(data['izz_wo'][data['stan_cywilny']==1],
                data['izz_wo'][data['stan_cywilny']==2])
# eta = Eta2(data, 'points', 'stan_cywilny', 4)

In [84]:
df['t'] = [t, '-',]
df['p'] = [p, '-',]
# df['eta2'] = [eta, '-', '-', '-']
df

,stan cywilny,M,SD,t,p
0,Panna,83.148148,11.561553,0.287875,0.77401
1,Mężatka,82.350000,12.737963,-,-


##Births

###Number

In [85]:
means = data[['porody','izz_wo']].groupby(['porody']).mean()
stds = data[['porody','izz_wo']].groupby(['porody']).std()

dict_temp = {
    'porody':['0 porodów','1 poród','2 porody','więcej niż 2 porody'],
    'M':means['izz_wo'].to_list(),
    'SD':stds['izz_wo'].to_list()
}

df = pd.DataFrame(dict_temp)
# df_age = df_age.drop([3])
df

,porody,M,SD
0,0 porodów,82.738095,12.315090
1,1 poród,81.612903,14.389064
2,2 porody,85.296296,12.865621
3,więcej niż 2 porody,81.400000,10.622079


In [87]:
f, p = f_oneway(data['izz_wo'][data['porody']==1],
                data['izz_wo'][data['porody']==2],
                data['izz_wo'][data['porody']==3],
                data['izz_wo'][data['porody']==4])
eta = Eta2(data, 'izz_wo', 'porody', 4)

In [88]:
df['F'] = [f, '-', '-', '-']
df['p'] = [p, '-', '-', '-']
df['eta2'] = [eta, '-', '-', '-']
df

,porody,M,SD,F,p,eta2
0,0 porodów,82.738095,12.315090,0.487072,0.691953,0.0127216
1,1 poród,81.612903,14.389064,-,-,-
2,2 porody,85.296296,12.865621,-,-,-
3,więcej niż 2 porody,81.400000,10.622079,-,-,-


###Types

In [89]:
means = data[['porody_typ','izz_wo']][data['porody'] != 1].groupby(['porody_typ']).mean()
stds = data[['porody_typ','izz_wo']][data['porody'] != 1].groupby(['porody_typ']).std()

dict_temp = {
    'porody_typ':['Tylko naturalne','Tylko cesarskie cięcie', 'inne'],
    'M':means['izz_wo'].to_list(),
    'SD':stds['izz_wo'].to_list()
}

df = pd.DataFrame(dict_temp)
df = df.drop([2])
df

,porody_typ,M,SD
0,Tylko naturalne,85.581395,12.167937
1,Tylko cesarskie cięcie,77.280000,13.947879


In [90]:
t, p = ttest_ind(data['izz_wo'][data['porody_typ']==1],
                data['izz_wo'][data['porody_typ']==2])
# eta = Eta2(data, 'points', 'stan_cywilny', 4)

In [91]:
df['t'] = [t, '-',]
df['p'] = [p, '-',]
# df['eta2'] = [eta, '-', '-', '-']
df

,porody_typ,M,SD,t,p
0,Tylko naturalne,85.581395,12.167937,2.56986,0.0124395
1,Tylko cesarskie cięcie,77.280000,13.947879,-,-


In [92]:
d = CohensD(data['izz_wo'][data['porody_typ']==1], data['izz_wo'][data['porody_typ']==2])

In [93]:
df['d'] = [d, '-',]
df

,porody_typ,M,SD,t,p,d
0,Tylko naturalne,85.581395,12.167937,2.56986,0.0124395,0.634266
1,Tylko cesarskie cięcie,77.280000,13.947879,-,-,-


#10 - RELATION BETWEEN IZZ AND KNOWLEDGE

In [94]:
corr_list = []

for scale_1 in ['izz_wo','izz_nawyki', 'izz_profi','izz_psych','izz_zdrow','points']:
  for scale_2 in ['izz_wo','izz_nawyki', 'izz_profi','izz_psych','izz_zdrow','points']:
    r = round(pearsonr(data[scale_1], data[scale_2])[0], 2)
    p = round(pearsonr(data[scale_1], data[scale_2])[1], 3)

    if str(p) == '0.0':
      p = 0.001
    
    corr_list.append(f'{r} ({p})')

In [95]:
corr_dict = {
    'scale':['izz_wo','izz_nawyki', 'izz_profi','izz_psych','izz_zdrow','points'],
    'izz_wo':corr_list[0:6],
    'izz_nawyki':corr_list[6:12],
    'izz_profi':corr_list[12:18],
    'izz_psych':corr_list[18:24],
    'izz_zdrow':corr_list[24:30],
    'points':corr_list[30:36],
}

df_cor = pd.DataFrame(corr_dict)

for c in df_cor.columns:
  for i in range(len(df_cor)):
    if df_cor[c][i] == '1.0 (0.001)':
      df_cor[c][i] = ''

print(f"Pearson's r correlations for used subscales (N={len(data)})")
print('r (p)')
print()

df_cor.to_excel('corr_matrix.xlsx')
df_cor

Pearson's r correlations for used subscales (N=115)
r (p)



,scale,izz_wo,izz_nawyki,izz_profi,izz_psych,izz_zdrow,points
0,izz_wo,,0.81 (0.001),0.81 (0.001),0.82 (0.001),0.77 (0.001),-0.01 (0.879)
1,izz_nawyki,0.81 (0.001),,0.65 (0.001),0.49 (0.001),0.48 (0.001),0.11 (0.225)
2,izz_profi,0.81 (0.001),0.65 (0.001),,0.55 (0.001),0.41 (0.001),0.18 (0.051)
3,izz_psych,0.82 (0.001),0.49 (0.001),0.55 (0.001),,0.56 (0.001),-0.22 (0.019)
4,izz_zdrow,0.77 (0.001),0.48 (0.001),0.41 (0.001),0.56 (0.001),,-0.11 (0.235)
5,points,-0.01 (0.879),0.11 (0.225),0.18 (0.051),-0.22 (0.019),-0.11 (0.235),
